In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
 
# Sample dataset (IMDb-like sentiment data)
texts = pd.read_csv("text.csv")['text'].values
labels = pd.read_csv("emotion_encoded.csv")['emotion_encoded'].values
 
# Tokenization and sequence conversion
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
print(sequences)
 
# Padding sequences
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
 
# Train Word2Vec model
w2v_model = Word2Vec(sentences=[text.split() for text in texts], vector_size=100, window=5, min_count=1, workers=4)
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
 
# Save embeddings to CSV
embedding_df = pd.DataFrame(embedding_matrix)
embedding_df.to_csv("word_embeddings.csv", index=False)
 
# Load embeddings from CSV
loaded_embedding_matrix = pd.read_csv("word_embeddings.csv").values
 
# Split data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
 
# Build RNN Model
model = Sequential([
    Embedding(vocab_size, 100, weights=[loaded_embedding_matrix], input_length=max_length, trainable=False),
    LSTM(128, activation='relu', return_sequences=False),
    Dense(7, activation='softmax')
])
 
# Compile and train the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, np.array(y_train), epochs=15, batch_size=2)
 
# Evaluate model

[[1905, 31, 206, 379], [52, 1272], [140, 315], [1906, 417, 611], [1906, 184, 155], [1906, 288, 143], [1906, 417, 379], [1906, 2159, 611], [1906, 288, 143], [1906, 391, 379, 291, 2024, 1250, 5481], [1906, 667, 536], [156, 2, 2584, 1572, 120, 249, 34], [288, 15, 10, 29, 2, 2680, 12, 4, 14157, 2484, 9, 1361, 5482], [10, 14158, 2, 251, 14159, 39, 26, 463, 3, 1362, 95, 458, 3, 2, 2, 47, 79, 2926, 18, 21, 1573], [10, 34, 9, 13, 4558], [10, 1172, 10, 48, 7, 270], [5, 51, 1, 59, 10, 9, 4, 1537, 20, 1, 59, 2, 10, 7090, 9, 2086, 36, 18, 68, 23, 7091], [1233, 4972, 19, 194, 255], [1861, 572, 90, 19, 2, 1498], [140, 4559, 175, 47, 12, 19, 351, 1611], [1, 132, 23, 115, 7, 17, 18, 19, 4560, 41, 40, 19, 1612, 23, 4191, 26, 45, 204, 5, 2379, 137], [352, 33, 1340], [209, 380, 10, 21, 68, 1862], [42, 72], [26, 35, 14, 223, 285, 2, 7092, 24, 11, 581, 104, 230, 2, 3895], [212, 34, 28, 5, 187, 212], [304, 612, 14160, 29, 1308], [139, 41, 31], [1341, 55], [382, 9, 842, 248, 1, 138, 1173, 548, 8, 488, 3438, 

/Users/denisbespalko/GitHub/2024-25c-fai2-adsai-group-team-28-y2c/.venv/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


20948/20948 ━━━━━━━━━━━━━━━━━━━━ 175s 8ms/step - accuracy: 0.4178 - loss: 1.4803
Epoch 2/15
 4742/20948 ━━━━━━━━━━━━━━━━━━━━ 2:10 8ms/step - accuracy: 0.4750 - loss: 1.4278

KeyboardInterrupt: 

In [9]:
print("X_train shape:", np.array(X_train).shape)
print("y_train shape:", np.array(y_train).shape)
print("X_test shape:", np.array(X_test).shape)
print("y_test shape:", np.array(y_test).shape)


X_train shape: (7, 6)
y_train shape: (7,)
X_test shape: (2, 6)
y_test shape: (2,)
